### Import the libraries

In [1]:
import boto3
import pandas as pd
import awswrangler as wr
from io import StringIO
from sagemaker import image_uris, TrainingInput
from sagemaker.estimator import Estimator
import datetime
import boto3
import awswrangler as wr
import pandas as pd
import sagemaker
from sagemaker.session import Session
from sagemaker.model_monitor import DataCaptureConfig
from sagemaker.model_monitor.dataset_format import DatasetFormat
from sagemaker.tuner import IntegerParameter, ContinuousParameter, HyperparameterTuner
from sagemaker.feature_store.feature_group import FeatureGroup
from io import StringIO
import datetime

sagemaker.config INFO - Not applying SDK defaults from location: /etc/xdg/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /home/sagemaker-user/.config/sagemaker/config.yaml


In [2]:
%store

Stored variables and their in-db values:
bucket_name                            -> 'housing-dataset-54355'
set_up_dependencies_passed             -> True
set_up_s3_bucket_passed                -> True


In [3]:
# get bucket_name
%store -r bucket_name
print(bucket_name)

housing-dataset-54355


In [4]:
# save Amazon information
account_id = boto3.client("sts").get_caller_identity().get("Account")
region = boto3.Session().region_name
role = sagemaker.get_execution_role()
sagemaker_session = sagemaker.Session()
s3 = boto3.client('s3', region_name=sagemaker_session.boto_region_name)

In [5]:
boto_session = boto3.Session(region_name=region)
sagemaker_client = boto_session.client(service_name='sagemaker', region_name=region)
featurestore_runtime = boto_session.client(service_name='sagemaker-featurestore-runtime', region_name=region)

feature_store_session = Session(
    boto_session=boto_session,
    sagemaker_client=sagemaker_client,
    sagemaker_featurestore_runtime_client=featurestore_runtime
)

### Construct Athena queries to read the data from each offline feature store

In [6]:
train_feature_group_name = "housing_train"
validation_feature_group_name = "housing_validation"
test_feature_group_name = "housing_test"

In [7]:
# set database name and table name
database_name = "housing"
table_name = "data"

In [8]:
sagemaker_client.list_feature_groups()

{'FeatureGroupSummaries': [{'FeatureGroupName': 'housing_validation',
   'FeatureGroupArn': 'arn:aws:sagemaker:us-east-1:453322325373:feature-group/housing_validation',
   'CreationTime': datetime.datetime(2024, 6, 13, 23, 56, 20, 615000, tzinfo=tzlocal()),
   'FeatureGroupStatus': 'Created',
   'OfflineStoreStatus': {'Status': 'Active'}},
  {'FeatureGroupName': 'housing_train',
   'FeatureGroupArn': 'arn:aws:sagemaker:us-east-1:453322325373:feature-group/housing_train',
   'CreationTime': datetime.datetime(2024, 6, 13, 23, 55, 58, 764000, tzinfo=tzlocal()),
   'FeatureGroupStatus': 'Created',
   'OfflineStoreStatus': {'Status': 'Active'}},
  {'FeatureGroupName': 'housing_test',
   'FeatureGroupArn': 'arn:aws:sagemaker:us-east-1:453322325373:feature-group/housing_test',
   'CreationTime': datetime.datetime(2024, 6, 13, 23, 56, 42, 62000, tzinfo=tzlocal()),
   'FeatureGroupStatus': 'Created',
   'OfflineStoreStatus': {'Status': 'Active'}}],
 'ResponseMetadata': {'RequestId': '094bfd5a-d

In [9]:
# Initialize FeatureGroup
train_feature_group = FeatureGroup(name=train_feature_group_name, sagemaker_session=sagemaker_session)

In [10]:
# Initialize FeatureGroup
validation_feature_group = FeatureGroup(name=validation_feature_group_name, sagemaker_session=sagemaker_session)

In [11]:
# Initialize FeatureGroup
test_feature_group = FeatureGroup(name=test_feature_group_name, sagemaker_session=sagemaker_session)

In [12]:
train_data_query = train_feature_group.athena_query()
validation_data_query = validation_feature_group.athena_query()
test_data_query = test_feature_group.athena_query()

In [13]:
train_data_table = train_data_query.table_name
validation_data_table = validation_data_query.table_name
test_data_table = test_data_query.table_name

In [14]:
train_query = f"""
SELECT * FROM "{train_data_table}"
"""

In [15]:
validation_query = f"""
SELECT * FROM "{validation_data_table}"
"""

In [16]:
test_query = f"""
SELECT * FROM "{test_data_table}"
"""

### Execute queries and put into dataframes

In [17]:
# run Athena query. The output is loaded to a Pandas dataframe.
df_train = pd.DataFrame()
train_data_query.run(query_string=train_query, output_location='s3://'+bucket_name+'/query_results/train/')
train_data_query.wait()
df_train = train_data_query.as_dataframe()

In [18]:
# run Athena query. The output is loaded to a Pandas dataframe.
df_validation = pd.DataFrame()
validation_data_query.run(query_string=validation_query, output_location='s3://'+bucket_name+'/query_results/validation/')
validation_data_query.wait()
df_validation = validation_data_query.as_dataframe()

In [19]:
# run Athena query. The output is loaded to a Pandas dataframe.
df_test = pd.DataFrame()
test_data_query.run(query_string=test_query, output_location='s3://'+bucket_name+'/query_results/test/')
test_data_query.wait()
df_test = test_data_query.as_dataframe()

### For df_train and df_validation the "id" column is removed and for df_test both the "id" column and "saleprice" column are removed

In [20]:
df_train = df_train.drop(['id', 'event_time', 'write_time','api_invocation_time','is_deleted'], axis=1)
df_validation = df_validation.drop(['id', 'event_time', 'write_time','api_invocation_time','is_deleted'], axis=1)
df_test = df_test.drop(['id', 'saleprice', 'event_time', 'write_time','api_invocation_time','is_deleted'], axis=1)

### Check the shape of the data

In [21]:
print(df_train.shape)
print(df_validation.shape)
print(df_test.shape)

(4086, 34)
(584, 34)
(292, 33)


In [22]:
df_train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4086 entries, 0 to 4085
Data columns (total 34 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   lotarea        4086 non-null   float64
 1   yearbuilt      4086 non-null   float64
 2   yearremodadd   4086 non-null   float64
 3   masvnrarea     4086 non-null   float64
 4   bsmtfinsf1     4086 non-null   float64
 5   bsmtfinsf2     4086 non-null   float64
 6   bsmtunfsf      4086 non-null   float64
 7   totalbsmtsf    4086 non-null   float64
 8   1stflrsf       4086 non-null   float64
 9   2ndflrsf       4086 non-null   float64
 10  lowqualfinsf   4086 non-null   float64
 11  grlivarea      4086 non-null   float64
 12  bsmtfullbath   4086 non-null   float64
 13  bsmthalfbath   4086 non-null   float64
 14  fullbath       4086 non-null   float64
 15  halfbath       4086 non-null   float64
 16  bedroomabvgr   4086 non-null   float64
 17  kitchenabvgr   4086 non-null   float64
 18  totrmsab

### Check the first few rows of the data

In [23]:
df_train.head(5)

,lotarea,yearbuilt,yearremodadd,masvnrarea,bsmtfinsf1,bsmtfinsf2,bsmtunfsf,totalbsmtsf,1stflrsf,2ndflrsf,...,openporchsf,enclosedporch,3ssnporch,screenporch,poolarea,miscval,mosold,yrsold,saleprice,logsaleprice
0,1879.0,1980.0,1980.0,0.0,366.0,0.0,150.0,516.0,516.0,516.0,...,0.0,0.0,0.0,0.0,0.0,0.0,12.0,2008.0,163000.0,12.001505
1,10380.0,1986.0,1987.0,172.0,28.0,1474.0,0.0,1502.0,1553.0,1177.0,...,96.0,0.0,0.0,0.0,0.0,0.0,8.0,2007.0,301000.0,12.614866
2,10637.0,2007.0,2008.0,336.0,1288.0,0.0,417.0,1705.0,1718.0,0.0,...,44.0,0.0,0.0,0.0,0.0,0.0,9.0,2009.0,297000.0,12.601487
3,10800.0,1914.0,1970.0,0.0,390.0,0.0,490.0,880.0,880.0,888.0,...,341.0,0.0,0.0,0.0,0.0,0.0,10.0,2006.0,163000.0,12.001505
4,9020.0,1964.0,1964.0,259.0,624.0,336.0,288.0,1248.0,1350.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,6.0,2008.0,168500.0,12.034691


In [24]:
df_validation.head(5)

,lotarea,yearbuilt,yearremodadd,masvnrarea,bsmtfinsf1,bsmtfinsf2,bsmtunfsf,totalbsmtsf,1stflrsf,2ndflrsf,...,openporchsf,enclosedporch,3ssnporch,screenporch,poolarea,miscval,mosold,yrsold,saleprice,logsaleprice
0,7200.0,1976.0,2001.0,0.0,288.0,0.0,396.0,684.0,684.0,714.0,...,0.0,0.0,0.0,0.0,0.0,0.0,2.0,2008.0,163000.0,12.001505
1,9382.0,1999.0,2000.0,125.0,0.0,0.0,1468.0,1468.0,1479.0,0.0,...,25.0,0.0,0.0,0.0,0.0,0.0,7.0,2008.0,191000.0,12.160029
2,8050.0,1947.0,1993.0,0.0,0.0,0.0,0.0,0.0,929.0,208.0,...,0.0,0.0,0.0,0.0,0.0,0.0,4.0,2008.0,163000.0,12.001505
3,2289.0,1978.0,1978.0,0.0,311.0,0.0,544.0,855.0,855.0,586.0,...,0.0,0.0,0.0,0.0,0.0,0.0,4.0,2009.0,148500.0,11.908340
4,13125.0,1991.0,1991.0,0.0,48.0,634.0,422.0,1104.0,912.0,1215.0,...,192.0,224.0,0.0,0.0,0.0,0.0,11.0,2007.0,238000.0,12.380026


In [25]:
df_test.head(5)

,lotarea,yearbuilt,yearremodadd,masvnrarea,bsmtfinsf1,bsmtfinsf2,bsmtunfsf,totalbsmtsf,1stflrsf,2ndflrsf,...,wooddecksf,openporchsf,enclosedporch,3ssnporch,screenporch,poolarea,miscval,mosold,yrsold,logsaleprice
0,5976.0,1920.0,1950.0,0.0,0.0,0.0,624.0,624.0,624.0,624.0,...,0.0,130.0,256.0,0.0,0.0,0.0,0.0,12.0,2006.0,12.001505
1,10421.0,1988.0,1988.0,42.0,394.0,0.0,586.0,980.0,980.0,734.0,...,228.0,66.0,156.0,0.0,0.0,0.0,500.0,3.0,2010.0,12.188418
2,13418.0,2006.0,2006.0,270.0,1420.0,0.0,430.0,1850.0,1850.0,898.0,...,212.0,182.0,0.0,0.0,0.0,0.0,0.0,10.0,2008.0,12.001505
3,7406.0,2006.0,2006.0,84.0,684.0,0.0,515.0,1199.0,1220.0,0.0,...,105.0,54.0,0.0,0.0,0.0,0.0,0.0,7.0,2006.0,12.175613
4,13770.0,1958.0,1998.0,340.0,190.0,873.0,95.0,1158.0,1176.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,10.0,2007.0,12.001505


### Save the data to the S3 bucket as CSV files

In [26]:
df_train.to_csv(f"s3://{bucket_name}/processed/housing/train.csv", header=False,  index=False)

In [27]:
df_validation.to_csv(f"s3://{bucket_name}/processed/housing/validation.csv", header=False, index=False)

In [28]:
df_test.to_csv(f"s3://{bucket_name}/processed/housing/test.csv", header=False, index=False)

### Train a model using SageMaker built-in XgBoost algorithm on the training data and validate it on the validation data
1. set the container image for the XGBoost algorithm
2. set the output path
3. set the hyperparameters
4. create an estimator
5. fit the model to the training data and validate on the validation data

In [29]:
# set the container image for the XGBoost algorithm
container = image_uris.retrieve(region=region, framework="xgboost", version="latest")

In [30]:
# set the output path
output_path = f"s3://{bucket_name}/housing/model"

In [31]:
# set the hyperparameters
hyperparameters = {
    "objective": "reg:linear",
    "num_round": "100",
    "max_depth": "5",
    "eta": "0.2",
    "gamma": "4",
    "min_child_weight": "6",
    "subsample": "0.7",
    "silent": "0",
}

In [32]:
# create an estimator
estimator = Estimator(
    image_uri=container,
    role=role,
    instance_count=1,
    instance_type="ml.m5.xlarge",
    output_path=output_path,
    hyperparameters=hyperparameters,
)

In [33]:
# fit the model to the training data and validate on the validation data
estimator.fit(
    {
        "train": TrainingInput(
            s3_data=f"s3://{bucket_name}/processed/housing/train.csv", content_type="text/csv"
        ),
        "validation": TrainingInput(
            s3_data=f"s3://{bucket_name}/processed/housing/validation.csv", content_type="text/csv"
        ),
    }
)

INFO:sagemaker:Creating training-job with name: xgboost-2024-06-14-18-55-59-706


2024-06-14 18:56:00 Starting - Starting the training job...
2024-06-14 18:56:17 Starting - Preparing the instances for training...
2024-06-14 18:56:48 Downloading - Downloading input data...
2024-06-14 18:57:08 Downloading - Downloading the training image...
2024-06-14 18:57:59 Training - Training image download completed. Training in progress.
2024-06-14 18:57:59 Uploading - Uploading generated training modelArguments: train
[2024-06-14:18:57:52:INFO] Running standalone xgboost training.
[2024-06-14:18:57:52:INFO] File size need to be processed in the node: 0.84mb. Available memory size in the node: 7996.05mb
[2024-06-14:18:57:52:INFO] Determined delimiter of CSV input is ','
[18:57:52] S3DistributionType set as FullyReplicated
[18:57:52] 4086x33 matrix with 134838 entries loaded from /opt/ml/input/data/train?format=csv&label_column=0&delimiter=,
[2024-06-14:18:57:52:INFO] Determined delimiter of CSV input is ','
[18:57:52] S3DistributionType set as FullyReplicated
[18:57:52] 584x33 m

### Upload the Sagemaker Model created during our training job to the Sagemaker Model Registry

In [34]:
# Get the current date and time
current_datetime = datetime.datetime.now().strftime("%Y-%m-%d-%H-%M-%S")
# Update the model_name variable
model_name = f"sagemaker-xgboost-{current_datetime}"
print(model_name)

sagemaker-xgboost-2024-06-14-18-58-42


In [36]:
# Saving training job information to be used in the ML lineage module
training_job_info = estimator.latest_training_job.describe()
if training_job_info != None:
    
    # Get the model data
    model_data = training_job_info["ModelArtifacts"]["S3ModelArtifacts"]
    # Create the primary container
    primary_container = {"Image": container, "ModelDataUrl": model_data}

    # Save our model to the Sagemaker Model Registry
    create_model_response = sagemaker_client.create_model(
        ModelName=model_name, ExecutionRoleArn=role, PrimaryContainer=primary_container
    )
    print(create_model_response["ModelArn"])

arn:aws:sagemaker:us-east-1:453322325373:model/sagemaker-xgboost-2024-06-14-18-58-42


In [37]:
# Inspect Training Job Details
training_job_info

{'TrainingJobName': 'xgboost-2024-06-14-18-55-59-706',
 'TrainingJobArn': 'arn:aws:sagemaker:us-east-1:453322325373:training-job/xgboost-2024-06-14-18-55-59-706',
 'ModelArtifacts': {'S3ModelArtifacts': 's3://housing-dataset-54355/housing/model/xgboost-2024-06-14-18-55-59-706/output/model.tar.gz'},
 'TrainingJobStatus': 'Completed',
 'SecondaryStatus': 'Completed',
 'HyperParameters': {'eta': '0.2',
  'gamma': '4',
  'max_depth': '5',
  'min_child_weight': '6',
  'num_round': '100',
  'objective': 'reg:linear',
  'silent': '0',
  'subsample': '0.7'},
 'AlgorithmSpecification': {'TrainingImage': '811284229777.dkr.ecr.us-east-1.amazonaws.com/xgboost:latest',
  'TrainingInputMode': 'File',
  'MetricDefinitions': [{'Name': 'train:mae',
    'Regex': '.*\\[[0-9]+\\].*#011train-mae:([-+]?[0-9]*\\.?[0-9]+(?:[eE][-+]?[0-9]+)?).*'},
   {'Name': 'train:merror',
    'Regex': '.*\\[[0-9]+\\].*#011train-merror:([-+]?[0-9]*\\.?[0-9]+(?:[eE][-+]?[0-9]+)?).*'},
   {'Name': 'validation:mae',
    'Regex'

### Host the trained XGBoost model as a SageMaker Endpoint

In [38]:
# deploy the model to an endpoint
predictor = estimator.deploy(
    initial_instance_count=1,
    instance_type="ml.m5.xlarge",
    endpoint_name="housing-endpoint",
    wait=True,
)

INFO:sagemaker:Creating model with name: xgboost-2024-06-14-19-04-11-800
INFO:sagemaker:Creating endpoint-config with name housing-endpoint
INFO:sagemaker:Creating endpoint with name housing-endpoint


------!

In [39]:
print(predictor.endpoint_name)

housing-endpoint


### Real time inference using the deployed endpoint

In [44]:
# make predictions using the endpoint
csv_data = df_test.to_csv(header=False, index=False)
predictions = predictor.predict(csv_data, initial_args={"ContentType": "text/csv"}).decode("utf-8")
predictions = pd.read_csv(StringIO(predictions), header=None)
predictions.head(5)

,0,1,2,3,4,5,6,7,8,9,...,282,283,284,285,286,287,288,289,290,291
0,17729.714844,16803.990234,35492.550781,23158.570312,26254.101562,18038.291016,25249.033203,44420.691406,21170.173828,19694.480469,...,30006.667969,28860.097656,28920.916016,24013.898438,17378.287109,20433.857422,36319.121094,20946.810547,21265.957031,17193.884766


### Delete the endpoint

In [45]:
# delete the endpoint
predictor.delete_endpoint()

INFO:sagemaker:Deleting endpoint configuration with name: housing-endpoint
INFO:sagemaker:Deleting endpoint with name: housing-endpoint


### Shut down notebook resources

In [46]:
%%javascript

try {
    Jupyter.notebook.save_checkpoint();
    Jupyter.notebook.session.delete();
}
catch(err) {
    // NoOp
}

<IPython.core.display.Javascript object>

In [47]:
%%html

<p><b>Shutting down your kernel for this notebook to release resources.</b></p>
<button class="sm-command-button" data-commandlinker-command="kernelmenu:shutdown" style="display:none;">Shutdown Kernel</button>
        
<script>
try {
    els = document.getElementsByClassName("sm-command-button");
    els[0].click();
}
catch(err) {
    // NoOp
}    
</script>